In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Media Company Case Study

## Multiple Linear Regression

### Problem Statement:
A digital media company (similar to Voot, Hotstar, Netflix, etc.) had launched a show. Initially, the show got a good response, but then witnessed a decline in viewership. The company wants to figure out what went wrong.


### Approach:
We are concerned about determining the driver variable for show viewership. This is the case of pridiction rather than projection where we are more interested in predicting the key driver variables and their impact rather than forcasting the results.

First we will list down the potential reasons for the decline in viewershp.<br>

The potential reasons could be:
1. Decline in the number of people coming to the platform
2. Fewer people watching the video
3. A Decrease in marketing spend?
4. Competitive shows, e.g. cricket/ IPL
5. Special holidays
6. Twist in the story


### Data
We have been given data for the period of 1 March 2017 to 19 May 2017.<br>
With Columns as<br> 
Views_show         : Number of times the show was viewed<br>
Visitors           : Number of visitors who browsed the platform, but not necessarily watched a video.<br>
Views_platform	   : Number of times a video was viewed on the platform<br>
Ad_impression	   : Proxy for marketing budget. Represents number of impressions generated by ads<br>
Cricket_match_india: If a cricket match was being played. 1 indicates match on a given day, 0 indicates there wasn't<br>
Character_A        : Describes presence of Character A. 1 indicates character A was in the episode, 0 indicates she/he wasn't

In [ ]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

# Import the numpy and pandas package

import numpy as np
import pandas as pd

# Data Visualisation

import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline

#miising value package

import missingno as msno

## Reading and Understanding the Data

In [ ]:
#Importing dataset
media = pd.read_csv("/kaggle/input/mediacompany.csv")
media.head()

In [ ]:
media.shape

In [ ]:
media.info()

In [ ]:
# Dropping the unwanted column
media = media.drop('Unnamed: 7',axis = 1)

In [ ]:
#checking duplicated rows presented in Dataframe or not
media[media.duplicated()]

In [ ]:
#Let's explore the top 5 rows
media.head()

In [ ]:
#lets check basic stats
media.describe()

In [ ]:
#Checking null values count
media.isnull().sum()

In [ ]:
#the best way to check null value by visualizing 
msno.bar(media)
#no null values

In [ ]:
#Target class 
sns.boxplot(media.Views_show)

In [ ]:
media.columns

In [ ]:
#Deselecting Date to know the ditribution of others
df = media.iloc[:,1:]
df.columns

In [ ]:
#Let us check the distribution graph for Views_show
ax= plt.figure(figsize=(10,5))
sns.distplot(df.Views_show, color='r')

In [ ]:
# Let us check the distribution graph for Visitors
ax= plt.figure(figsize=(10,5))
sns.distplot(df.Visitors, color='g')

In [ ]:
# Let us check the distribution graph for Views_platform
ax= plt.figure(figsize=(10,5))
sns.distplot(df.Views_platform, color='y')

In [ ]:
# Let us check the box graph for Views_show
ax= plt.figure(figsize=(10,5))
sns.boxplot(df.Views_show, color='peru')

In [ ]:
# Let us check the box graph for Visitors
ax= plt.figure(figsize=(10,5))
sns.boxplot(df.Visitors, color='r')

In [ ]:
# Let us check the box graph for Views_platform
ax= plt.figure(figsize=(10,5))
sns.boxplot(df.Views_platform, color='g')

In [ ]:
# Let us check the box graph for Ad_impression
ax= plt.figure(figsize=(10,5))
sns.boxplot(df.Ad_impression, color='b')

boxplot to check outliers
in the above plot we found minimal amount of outliers so we dont need to worry about them

In [ ]:
boxplot to check outliers
in the above plot we found minimal amount of outliers so we dont need to worry about them

In [ ]:
#converting into Timestamp format
media['Date'] = pd.to_datetime(media['Date'], dayfirst = False )

In [ ]:
media.tail()

In [ ]:
media['Day_of_week'] = media['Date'].dt.dayofweek
media['month'] = media['Date'].dt.month
media['day'] = media['Date'].dt.day

In [ ]:
media.head()

In [ ]:
#lets check Views_show vs Day_of_week
sns.barplot(data = media,x='Day_of_week', y='Views_show')

In [ ]:
#mapping a dict to make it readable
di = {0:'Mo', 1:'Tu', 2:'We', 3:'Th', 4:'Fr', 5:'Sa', 6:'Su'}

media['day_name'] = media['Day_of_week'].map(di)

In [ ]:
#lets make it more beatuiful plot in orderwise
plot_order = media.groupby('day_name')['Views_show'].sum().sort_values(ascending=False).index.values

sns.catplot(data=media, x='day_name', y='Views_show',kind='bar',ci=None, legend_out=False, order=plot_order)

fig = plt.gcf()
fig.set_size_inches(10, 5)

In [ ]:
#lets check Views_show vs day
plot_order1 = media.groupby('day')['Views_show'].mean().sort_values(ascending=False).index.values

sns.catplot(data=media, x='day', y='Views_show',kind='bar',ci=None, legend_out=False, order=plot_order1)

fig = plt.gcf()
fig.set_size_inches(16, 5)

In [ ]:
#lets map it 
# 0 : week days, 1 : weekend day
wd = {0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1}
media['weekend'] = media['Day_of_week'].map(wd)

In [ ]:
sns.barplot(data=media, x='weekend', y='Views_show')
fig = plt.gcf()
fig.set_size_inches(5, 5)

In [ ]:
media.head()

In [ ]:
sns.lmplot(data = media,x='Visitors', y='Views_platform')#,hue='Cricket_match_india')
fig = plt.gcf()
fig.set_size_inches(10, 5)

In [ ]:
sns.scatterplot(data = media,x='Visitors', y='Views_show', hue='Character_A')
fig = plt.gcf()
fig.set_size_inches(10, 5)

#Character_A : 1:appearance, 0.No appearance

In [ ]:
sns.scatterplot(data = media,x='Visitors', y='Views_show',hue='Cricket_match_india')
fig = plt.gcf()
fig.set_size_inches(10, 5)

#Cricket_match_india: 1: Match , 0:No Match

In [ ]:
media.head()

In [ ]:
(media.iloc[:,1:5]).plot(figsize=(8,16),subplots=True)

In [ ]:
# plot for Date vs Views_show and days vs Ad_impressions
ax = media.plot(x="Date", y="Views_show", legend=False)
ax2 = ax.twinx()
media.plot(x="Date", y="Ad_impression", ax=ax2, legend=False, color="r")
ax.figure.legend()

In [ ]:
sns.pairplot(media)

In [ ]:
media.head(5)

In [ ]:
df = media.iloc[:,1:5]
df

# Model Building

In [ ]:
X = df.iloc[:,1:]
X

y = df['Views_show']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['Views_show','Visitors','Views_platform','Ad_impression']

media[num_vars] = scaler.fit_transform(media[num_vars])

In [ ]:
media.head()

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated

In [ ]:
sns.heatmap(media.corr(),annot = True)

#### Running first model (lm1) Visitors, weekend

In [ ]:
# Putting feature variable to X
X = media[['Visitors','weekend']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Representing LinearRegression as lm(Creating LinearRegression Object)
lm = LinearRegression()

In [ ]:
# fit the model to the training data
lm.fit(X,y)

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_1 = sm.OLS(y,X).fit()
print(lm_1.summary())

In [ ]:
# Inference:
# Visitors as well as weekend column are significant.

#### Running second model (lm2) visitors, weekend & Character_A

In [ ]:
# Putting feature variable to X
X = media[['Visitors','weekend','Character_A']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_2 = sm.OLS(y,X).fit()
print(lm_2.summary())

In [ ]:
# we have seen that views of today effects views of tomorrow. So to take that in account we will create a Lag variable.

In [ ]:
# Create lag variable
media['Lag_Views'] = np.roll(media['Views_show'], 1)
media.head()

In [ ]:
media.Lag_Views[0] = 0

In [ ]:
media.head()

#### Running third model (lm3) visitors, Character_A, Lag_views & weekend

In [ ]:
# Putting feature variable to X
X = media[['Visitors','Character_A','Lag_Views','weekend']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_3 = sm.OLS(y,X).fit()
print(lm_3.summary())

In [ ]:
# Inference:
# It leaves visitor insignificant.

#### Running fourth model (lm4) Character_A, weekend & Views_platform

In [ ]:
# Putting feature variable to X
X = media[['weekend','Character_A','Views_platform']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_4 = sm.OLS(y,X).fit()
print(lm_4.summary())

#### Running fifth model (lm5) Character_A, weekend & Visitors

In [ ]:
# Putting feature variable to X
X = media[['weekend','Character_A','Visitors']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_5 = sm.OLS(y,X).fit()
print(lm_5.summary())

#### Running sixth model (lm6) Character_A, weekend, Visitors & Ad_impressions

In [ ]:
# Putting feature variable to X
X = media[['weekend','Character_A','Visitors','Ad_impression']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_6 = sm.OLS(y,X).fit()
print(lm_6.summary())

#### Running seventh model (lm7) Character_A, weekend & Ad_impressions

In [ ]:
# Inference
# we can observe a pattern in the plot.

In [ ]:
# Putting feature variable to X
X = media[['weekend','Character_A','Ad_impression']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_7 = sm.OLS(y,X).fit()
print(lm_7.summary())

In [ ]:
#Ad impression in million
media['ad_impression_million'] = media['Ad_impression']/1000000

#### Running seventh model (lm8) Character_A, weekend, Visitors, ad_impressions_million & Cricket_match_india

In [ ]:
# Putting feature variable to X
X = media[['weekend','Character_A','ad_impression_million','Cricket_match_india']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_8 = sm.OLS(y,X).fit()
print(lm_8.summary())

#### Running seventh model (lm9) Character_A, weekend & ad_impressions_million

In [ ]:
# Putting feature variable to X
X = media[['weekend','Character_A','ad_impression_million']]

# Putting response variable to y
y = media['Views_show']

In [ ]:
import statsmodels.api as sm
#Unlike SKLearn, statsmodels don't automatically fit a constant, 
#so you need to use the method sm.add_constant(X) in order to add a constant. 
X = sm.add_constant(X)
# create a fitted model in one line
lm_9 = sm.OLS(y,X).fit()
print(lm_9.summary())

#### Making predictions using lm 9 

In [ ]:
# Making predictions using the model
X = media[['weekend','Character_A','ad_impression_million']]
X = sm.add_constant(X)
Predicted_views = lm_9.predict(X)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(media.Views_show, Predicted_views)
r_squared = r2_score(media.Views_show, Predicted_views)

In [ ]:
print('Mean_Squared_Error :' ,mse)
print('r_square_value :',r_squared)

In [ ]:
#Actual vs Predicted
c = [i for i in range(1,81,1)]
fig = plt.figure()
plt.plot(c,media.Views_show, color="blue", linewidth=2.5, linestyle="-")
plt.plot(c,Predicted_views, color="red",  linewidth=2.5, linestyle="-")
fig.suptitle('Actual and Predicted', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                               # X-label
plt.ylabel('Views', fontsize=16)                               # Y-label

In [ ]:
# Error terms
c = [i for i in range(1,81,1)]
fig = plt.figure()
plt.plot(c,media.Views_show-Predicted_views, color="blue", linewidth=2.5, linestyle="-")
fig.suptitle('Error Terms', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                      # X-label
plt.ylabel('Views_show-Predicted_views', fontsize=16)                # Y-label

#### Making predictions using lm5

In [ ]:
# Making predictions using the model
X = media[['weekend','Character_A','Visitors']]
X = sm.add_constant(X)
Predicted_views = lm_5.predict(X)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(media.Views_show, Predicted_views)
r_squared = r2_score(media.Views_show, Predicted_views)

In [ ]:
print('Mean_Squared_Error :' ,mse)
print('r_square_value :',r_squared)

In [ ]:
#Actual vs Predicted
c = [i for i in range(1,81,1)]
fig = plt.figure()
plt.plot(c,media.Views_show, color="blue", linewidth=2.5, linestyle="-")
plt.plot(c,Predicted_views, color="red",  linewidth=2.5, linestyle="-")
fig.suptitle('Actual and Predicted', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                               # X-label
plt.ylabel('Views', fontsize=16)                               # Y-label

In [ ]:
# Error terms
c = [i for i in range(1,81,1)]
fig = plt.figure()
plt.plot(c,media.Views_show-Predicted_views, color="blue", linewidth=2.5, linestyle="-")
fig.suptitle('Error Terms', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                      # X-label
plt.ylabel('Views_show-Predicted_views', fontsize=16)                # Y-label

 Ad Impressions and Character A as the driver variables that could explain the
viewership pattern. Based on industry experience, ad impressions are directly proportional to the
marketing budget. Thus, by increasing the marketing budget, a better viewership could be
achieved. Similarly, Character A’s absence and presence created a significant change in show viewership.
Character A’s presence brings viewers to the show. Thus, these two variables could be acted upon to
improve show viewership.